In [1]:
"""
Created on Mon May 13 11:45:31 2024

The goal of this script is to use one mhd file to map a region that will then be used by medtool to create mesh

@author: cofo
"""
import SimpleITK as sitk
import tkinter as tk
from tkinter import filedialog
import numpy as np
import ipywidgets
import matplotlib.pyplot as plt
from PIL import Image
import scipy
import os
import shutil
import pandas as pd

import sys
# Add the directory to the sys.path
sys.path.append('D:/EmpaData/COFO - HP/03 - Github/dvpt-at-EMPA/HappyBisons/image analysis')
# Now you can import modules from that directory
import tools.CT_modification_functions as CT_modification_functions
import tools.CT_visualization_functions as CT_visualization_functions
import tools.CT_visualization_window as CT_visualization_window
%matplotlib qt


In [2]:
#============ setting up work direction ===============

current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
gr_parent_dir = os.path.dirname(parent_dir)
gr_gr_parent_dir = os.path.dirname(gr_parent_dir)

print(current_dir)
print(parent_dir)
print(gr_parent_dir)
print(gr_gr_parent_dir)

input_file_path = gr_gr_parent_dir + "\\02 - Images Processing\\023 - filtering\\filtered_image-set1.mhd"
output_folder_path = current_dir

print(input_file_path)
print(output_folder_path)


d:\EmpaData\COFO - HP\01 PhD research\202406 - Basel skulls\WP4 - skull 11152\03 - Simulation Setup\031 - create mesh volume\python
d:\EmpaData\COFO - HP\01 PhD research\202406 - Basel skulls\WP4 - skull 11152\03 - Simulation Setup\031 - create mesh volume
d:\EmpaData\COFO - HP\01 PhD research\202406 - Basel skulls\WP4 - skull 11152\03 - Simulation Setup
d:\EmpaData\COFO - HP\01 PhD research\202406 - Basel skulls\WP4 - skull 11152
d:\EmpaData\COFO - HP\01 PhD research\202406 - Basel skulls\WP4 - skull 11152\02 - Images Processing\023 - filtering\filtered_image-set1.mhd
d:\EmpaData\COFO - HP\01 PhD research\202406 - Basel skulls\WP4 - skull 11152\03 - Simulation Setup\031 - create mesh volume\python


In [3]:
#============ loading files ===============

# Load the image from file
image = sitk.ReadImage(input_file_path)
smoothed_image = sitk.SmoothingRecursiveGaussian(image, sigma=0.4)

# Convert the SimpleITK image to a NumPy array
image_array = sitk.GetArrayFromImage(image)
image_array_smoothed = sitk.GetArrayFromImage(smoothed_image)

#see it
CT_visualization_functions.browse_images(image)




In [46]:
#============= transform all image into binary: either there is nothing (anything) or not ==============

#thresholding
binary_image = CT_modification_functions.segment_from_threshold(smoothed_image,100,2200)

binary_image = CT_modification_functions.connected_filter(287,353,1308,binary_image)

#binary_image = CT_modification_functions.erode_filter(binary_image,1)

# Convert the SimpleITK image to a NumPy array
image_array_binary = sitk.GetArrayFromImage(binary_image) 

#see it
CT_visualization_functions.browse_blended_images(image,binary_image,True)


 =================================================================================================================
 ============================================ first mask : wood ==================================================
 =================================================================================================================

In [5]:
# =========== making a first mask to segment the wood in the images =================


mask1_full = CT_modification_functions.segment_from_threshold(image,100,600)

CT_visualization_window.open_window(mask1_full)
#CT_visualization_functions.browse_blended_images(mask1_full,binary_image,False)

array([], dtype=float64)

In [9]:
#============ edit the first mask ===============

#imaging_functions.browse_images(mask1_full)
mask1 = CT_modification_functions.dilate_filter(mask1_full,1)
#imaging_functions.browse_images(mask1)
mask1 = CT_modification_functions.erode_filter(mask1,2)
#imaging_functions.browse_images(mask1)

mask1 = CT_modification_functions.connected_filter(518,556,928,mask1)

mask1 = CT_modification_functions.dilate_filter(mask1,3)

mask1 = CT_modification_functions.dilate_filter(mask1,3)

mask1 = CT_modification_functions.dilate_filter(mask1,2)

mask1 = CT_modification_functions.erode_filter(mask1,5)


CT_visualization_functions.browse_blended_images(image,mask1,True)
CT_visualization_functions.browse_blended_images(mask1,binary_image,True)

In [19]:
#============ manually edit the first mask ===============
# !!!!! DO NOT RE-RUN !!!!!
out = CT_visualization_functions.select_pixels(mask1)
np.savetxt(output_folder_path + '/mask_wood.txt', out, fmt='%d', delimiter=',', header='x,y,slice', comments='', newline='\n')
CT_visualization_functions.browse_blended_images(image,mask1,True)

In [12]:
CT_visualization_window.open_window(mask1)

Clicked at coordinates: x=522, y=541, in slice 0, Pixel value: 255.0


array([], dtype=float64)

In [18]:
#============ load the manual mask and add it to the first mask ===============


manual_mask_path = output_folder_path + '/mask_wood.txt'

def load_data_as_numpy_array(file_path):
    # Read the data from the file
    data = pd.read_csv(file_path, sep=',', header=0)
    # Convert the list of tuples into a numpy array
    return np.array(data)


manual_mask_array = load_data_as_numpy_array(manual_mask_path)

mask1_f_array = sitk.GetArrayFromImage(mask1)

for row in manual_mask_array:
    mask1_f_array[row[2],row[1],row[0]] = 0 #numpy array is a pain so numbers are reversed

mask1_f = sitk.GetImageFromArray(mask1_f_array)

mask1_f = CT_modification_functions.connected_filter(522,541,0,mask1_f)

CT_visualization_functions.browse_blended_images(mask1_f,image,True)


In [19]:
#============ save the resulting mask ===============

sitk.WriteImage(mask1_f, output_folder_path + "/mask1.mhd")

In [47]:
#============== applying first mask to binary image to remove wood ==================
mask1_array = sitk.GetArrayFromImage(mask1_f)

new_image_array_binary = image_array_binary
new_image_array_binary[mask1_array > 0] = 0

new_image_binary = sitk.GetImageFromArray(new_image_array_binary)
CT_visualization_functions.browse_images(new_image_binary)

new_image_binary = CT_modification_functions.erode_filter(new_image_binary,1)

new_image_binary = CT_modification_functions.connected_filter(407,90,1032,new_image_binary)
CT_visualization_functions.browse_images(new_image_binary)

image_binary_2 = CT_modification_functions.dilate_filter(new_image_binary,1)
CT_visualization_functions.browse_images(image_binary_2)




 =================================================================================================================
 ========================================= second mask : shiny block =============================================
 =================================================================================================================

In [65]:
array_image_binary_2 = sitk.GetArrayFromImage(image_binary_2)
mask2 = sitk.GetImageFromArray(array_image_binary_2)
array_mask_2 = sitk.GetArrayFromImage(image_binary_2)

In [68]:
#============ start second mask ===============

x = 400
y= 380
z = 1425
xrange = 140
yrange = 100
zrange = 200

temp_array = sitk.GetArrayFromImage(image)

for i in range(x,x+xrange):
    for j in range(y,y+yrange):
        for k in range(z,z+zrange):
            array_mask_2[k,j,i] = 0 #numpy array is a pain
            temp_array[k,j,i] = 0

temp_image = sitk.GetImageFromArray(temp_array)

mask2 = sitk.GetImageFromArray(array_mask_2)
mask2 = CT_modification_functions.connected_filter(340,270,1052,mask2)

CT_visualization_functions.browse_blended_images(mask2,image,True)
#CT_visualization_functions.browse_images(mask2)
mask2_array = sitk.GetArrayFromImage(mask2)




In [69]:
#============ manually edit the first mask ===============
# !!!!! DO NOT RE-RUN !!!!!
out = CT_visualization_window.open_window(temp_image)
np.savetxt(output_folder_path + '/mask_2_1.txt', out, fmt='%d', delimiter=',', header='x,y,slice', comments='', newline='\n')
CT_visualization_functions.browse_blended_images(image,mask1,True)

Clicked at coordinates: x=536, y=386, in slice 1434, Pixel value: 0.0
Clicked at coordinates: x=538, y=393, in slice 1434, Pixel value: 0.0
Clicked at coordinates: x=540, y=394, in slice 1434, Pixel value: 306.0
Clicked at coordinates: x=542, y=394, in slice 1434, Pixel value: 366.0
Clicked at coordinates: x=546, y=393, in slice 1434, Pixel value: 281.0
Clicked at coordinates: x=552, y=391, in slice 1434, Pixel value: 98.0
Clicked at coordinates: x=492, y=381, in slice 1443, Pixel value: 0.0
Clicked at coordinates: x=497, y=378, in slice 1443, Pixel value: 671.0
Clicked at coordinates: x=508, y=377, in slice 1443, Pixel value: 237.0
Clicked at coordinates: x=532, y=379, in slice 1443, Pixel value: 187.0
Clicked at coordinates: x=533, y=379, in slice 1443, Pixel value: 221.0
Clicked at coordinates: x=539, y=373, in slice 1443, Pixel value: 19.0
Clicked at coordinates: x=521, y=369, in slice 1443, Pixel value: 186.0
Clicked at coordinates: x=511, y=373, in slice 1443, Pixel value: 112.0


In [70]:
#============ load the manual mask and add it to the first mask ===============


manual_mask_path = output_folder_path + '/mask_2_1.txt'

def load_data_as_numpy_array(file_path):
    # Read the data from the file
    data = pd.read_csv(file_path, sep=',', header=0)
    # Convert the list of tuples into a numpy array
    return np.array(data)


manual_mask_array = load_data_as_numpy_array(manual_mask_path)

for row in manual_mask_array:
    array_mask_2[row[2],row[1],row[0]] = 0 #numpy array is a pain so numbers are reversed

mask_2 = sitk.GetImageFromArray(array_mask_2)

CT_visualization_functions.browse_blended_images(mask_2,image,True)

In [77]:
CT_visualization_window.open_window(image_binary_2)

Clicked at coordinates: x=469, y=96, in slice 1348, Pixel value: 255.0


array([], dtype=float64)

In [78]:
#============ apply the second mask to the image ===============
image_binary_3 = image_binary_2
array_image_binary_3 =sitk.GetArrayFromImage(image_binary_3)
array_image_binary_3[array_mask_2 == 0] = 0

image_binary_3 = sitk.GetImageFromArray(array_image_binary_3)
#image_binary_3 = CT_modification_functions.connected_filter(469,96,1348,image_binary_3)

CT_visualization_functions.browse_images(image_binary_3)

In [79]:
#============ save the second mask ===============
sitk.WriteImage(mask2, output_folder_path + "/mask2.mhd")

 =================================================================================================================
 ============================================ third mask : tiny shit =============================================
 =================================================================================================================

In [85]:

CT_visualization_functions.browse_blended_images(image_binary_3, image, True)

In [86]:

array_image_binary_3 = sitk.GetArrayFromImage(image_binary_3)


In [87]:
# !!!!! DO NOT RE-RUN !!!!!
out = CT_visualization_window.open_window(image_binary_3)
selected_pixels_array = np.array(out)
np.savetxt(output_folder_path + '/mask3_1.txt', selected_pixels_array, fmt='%d', delimiter=',', header='x,y,slice', comments='', newline='\n')


Clicked at coordinates: x=411, y=449, in slice 669, Pixel value: 0.0
Clicked at coordinates: x=407, y=440, in slice 669, Pixel value: 255.0
Clicked at coordinates: x=407, y=442, in slice 677, Pixel value: 255.0
Clicked at coordinates: x=407, y=443, in slice 685, Pixel value: 0.0
Clicked at coordinates: x=476, y=448, in slice 720, Pixel value: 0.0
Clicked at coordinates: x=476, y=448, in slice 725, Pixel value: 255.0
Clicked at coordinates: x=477, y=454, in slice 730, Pixel value: 0.0
Clicked at coordinates: x=382, y=427, in slice 944, Pixel value: 255.0
Clicked at coordinates: x=380, y=427, in slice 963, Pixel value: 0.0
Clicked at coordinates: x=380, y=422, in slice 969, Pixel value: 0.0
Clicked at coordinates: x=380, y=422, in slice 978, Pixel value: 0.0
Clicked at coordinates: x=374, y=418, in slice 981, Pixel value: 0.0
Clicked at coordinates: x=369, y=416, in slice 986, Pixel value: 255.0
Clicked at coordinates: x=372, y=414, in slice 994, Pixel value: 0.0
Clicked at coordinates: 

In [90]:
CT_visualization_functions.browse_images(image_binary_3)

mask3 = image_binary_3
array_mask3 = sitk.GetArrayFromImage(image_binary_3)

mask_31_path = output_folder_path + '/mask3_1.txt'
mask_31_array = load_data_as_numpy_array(mask_31_path)



for row in mask_31_array:
    array_mask3[row[2],row[1],row[0]] = 0 #numpy array is a pain


mask3 = sitk.GetImageFromArray(array_mask3)
CT_visualization_functions.browse_images(mask3)


mask3 = CT_modification_functions.connected_filter(390,90,1052,mask3)
array_mask3 = sitk.GetArrayFromImage(mask3)


CT_visualization_functions.browse_blended_images(mask3,image_binary_3,True)



In [93]:

image_binary_4 = image_binary_3
array_image_binary_4 =sitk.GetArrayFromImage(image_binary_4)
array_image_binary_4[array_mask3 == 0] = 0

image_binary_4 = sitk.GetImageFromArray(array_image_binary_4)
CT_visualization_functions.browse_images(image_binary_4)

In [94]:
sitk.WriteImage(mask3, output_folder_path + "/mask3.mhd")

In [95]:
sitk.WriteImage(image_binary_4, output_folder_path + "/map_whole_meshing_region.mhd")

In [96]:
print(output_folder_path)

d:\EmpaData\COFO - HP\01 PhD research\202406 - Basel skulls\WP4 - skull 11152\03 - Simulation Setup\031 - create mesh volume\python
